# Question 3: Hotel La Quinta Motor Inns (LQM)

La Quinta Motors are planning to purchase some hotels in California. They have a budget of 10 million. They want to purchase hotels that would yield them maximum profit. 

Hotels have different prices and profitabilities. We need to suggest an optimal solution LQM that suggests which hotels they should buy maximising their profit.

### Import libraries

In [0]:
import pandas as pd
import numpy as np
import math
import cvxpy as cvx
from numpy import matrix 
from cvxpy import *
from prettytable import PrettyTable

### Reading input from file

Using the below formula to calculate profitability.

𝑃𝑟𝑜𝑓𝑖𝑡𝑎𝑏𝑖𝑙𝑖𝑡𝑦 = 39.05 − 5.41×𝑆𝑡𝑎𝑡𝑒 𝑃𝑜𝑝𝑢𝑙𝑎𝑡𝑖𝑜𝑛 𝑝𝑒𝑟 𝐼𝑛𝑛 + 
5.86×𝑃𝑟𝑖𝑐𝑒 𝑜𝑓 𝑡ℎ𝑒 𝐼𝑛𝑛
− 3.09×𝑆𝑞𝑢𝑎𝑟𝑒 𝑅𝑜𝑜𝑡 𝑜𝑓 𝑡ℎ𝑒 𝑀𝑒𝑑𝑖𝑎𝑛 𝐼𝑛𝑐𝑜𝑚𝑒 𝑖𝑛 𝑡ℎ𝑒 𝑎𝑟𝑒𝑎 + 
1.75×𝐶𝑜𝑙𝑙𝑒𝑔𝑒 𝑆𝑡𝑢𝑑𝑒𝑛𝑡𝑠 𝑖𝑛 𝑡ℎ𝑒 𝐴𝑟𝑒𝑎
                                    

Profitability is calculated in excel sheet that is attached. The rows are sorting with the profitability value in their descending order.


In [0]:
url = 'https://raw.github.com/krithikaceg/Optimization/master/SelectingHotels%20%20MSBA.xlsx'
input_df = pd.read_excel(url)
hotel_list = input_df['Hotel'].tolist()
price_list = input_df['Price'].tolist()
profitability_list = input_df['Profitability'].tolist()
city_list =  input_df['Location'].tolist()

# creating cvx variable
x = cvx.Variable(16, boolean=True) # define a vector of n Boolean variables, i.e., x_i={0,1}  

### Data computed from input file

In [0]:
input_df

,Hotel,Location,Price,Price (normalized),Square Root of Median Income (normalized),College Students in Area (normalized),State Population Per Inn (normalized),Profitability
0,2,"Fresno, California",10000000,1.699076,-0.408199,0.311669,-0.474279,53.38
1,6,"Long Beach, California",8950000,1.402123,0.657845,0.483711,-0.557278,49.10
2,1,"Eureka, California",2925000,-0.301823,-0.812780,-0.536413,-0.995987,44.24
3,3,"Fresno, California",3750000,-0.068503,-0.408199,0.311669,-0.474279,43.02
4,4,"Fresno, California",3500000,-0.139206,-0.408199,0.311669,-0.474279,42.61
5,14,"South Lake Tahoe, California",3750000,-0.068503,-0.791006,-0.594905,-0.426851,42.36
6,12,"South Lake Tahoe, California",2500000,-0.422019,-0.791006,-0.594905,-0.426851,40.29
7,13,"South Lake Tahoe, California",1975000,-0.570495,-0.791006,-0.594905,-0.426851,39.42
8,10,"South Lake Tahoe, California",1650000,-0.662409,-0.791006,-0.594905,-0.426851,38.88
9,15,"South Lake Tahoe, California",1475000,-0.711902,-0.791006,-0.594905,-0.426851,38.59


### Variables affecting profitability

Part 1.

From the above equation, 𝑃𝑟𝑖𝑐𝑒 𝑜𝑓 𝑡ℎ𝑒 𝐼𝑛𝑛 affects profitability positively, whereas 𝑆𝑡𝑎𝑡𝑒 𝑃𝑜𝑝𝑢𝑙𝑎𝑡𝑖𝑜𝑛 𝑝𝑒𝑟 𝐼𝑛𝑛 affect profotability negatively. While they make the biggest impact, other variables such as 𝐶𝑜𝑙𝑙𝑒𝑔𝑒 𝑆𝑡𝑢𝑑𝑒𝑛𝑡𝑠 𝑖𝑛 𝑡ℎ𝑒 𝐴𝑟𝑒𝑎 affects profotability positively, while 𝑆𝑞𝑢𝑎𝑟𝑒 𝑅𝑜𝑜𝑡 𝑜𝑓 𝑡ℎ𝑒 𝑀𝑒𝑑𝑖𝑎𝑛 𝐼𝑛𝑐𝑜𝑚𝑒 𝑖𝑛 𝑡ℎ𝑒 𝑎𝑟𝑒𝑎 affect the profitability negatively.

𝑃𝑟𝑖𝑐𝑒 𝑜𝑓 𝑡ℎ𝑒 𝐼𝑛𝑛 can directly contribute to the profit as long it is not over-priced. 

State population per inn = State population/ number of inns in that place.

Since all of these hotels are in California, state population is a constant value across all the hotels in California state. So, it is the number of inns in that place that distinguishes one place from the other. (Hotels belonging to the same place has same State population per inn value.) Hence, if the state population per inn value is more, the number of inns is less in that place.

The Number of inns in that place increases with increase in people staying in that place. When the number of inns in the state is less, that implies the number of people staying is less, hence the profitability is less and vice versa.



Part 2.
Based on the formula used in the excel, the highest profitability is from Hotel #2, Fresno, California, whereas the lowest profitability is from Hotel #8, Los Angeles, California.

In [0]:
print("Maximum Profitable hotel: %s" %hotel_list[np.argmax(profitability_list)])
#print(hotel_list[np.argmax(profitability_list)])
print("Minimum Profitable hotel: %s" %hotel_list[np.argmin(profitability_list)])

Maximum Profitable hotel: 2
Minimum Profitable hotel: 8


Part 3:
Using the greedy approach, we choose from the most profitable hotel until we run out of budget. 

In that case, if we are given 10million, we would choose only one hotel: Hotel #2.

Sum of predicted profitability is 53.38

If we are trying to maximize our total predicted profitability, is this "not" a good approach. If we were trying to maximise the average profitability of the hotels purchsed, we would want to minimise the variance between the profitability of the hotels.

If we were given 20 million, the hotels purchased yielding higher total predicted profitability would be, hotels numbers: 0, 1, 11

In [0]:
budget = 20000000
hotels = []
for i in range(len(price_list)):
  balance_budget = budget - price_list[i]
  if (balance_budget > 0):
    hotels.append(i)
    budget = balance_budget
print(hotels)

[0, 1, 11]


### Constraints

In [0]:
c1 = sum(price_list*x) <= 10000000
con = [c1]

### Objective Function

In [0]:
obj1 = cvx.Maximize(sum(profitability_list*x))

### Maximum Profitability

In [0]:
prob = cvx.Problem(obj1, con)
result = prob.solve()

print('Maximum profitability')
print(round(prob.value))

Maximum profitability
270.0


### Hotels Chosen

In [0]:
np.around(x.value)

array([-0., -0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  0.])

In [0]:
pt = PrettyTable()

print('Hotels that are chosen')

pt.field_names = ["Hotel", "City"]
for i in range(16):
  if (x[i].value > 0.99):
    pt.add_row([hotel_list[i], city_list[i]])

print(pt)

Hotels that are chosen
+-------+------------------------------+
| Hotel |             City             |
+-------+------------------------------+
|   12  | South Lake Tahoe, California |
|   13  | South Lake Tahoe, California |
|   10  | South Lake Tahoe, California |
|   15  | South Lake Tahoe, California |
|   11  | South Lake Tahoe, California |
|   16  | South Lake Tahoe, California |
|   5   |      Fresno, California      |
+-------+------------------------------+


Yes, it definitely is better than greedy solution. While we are trying to stick within our budget, we are also trying to find the optimal solution that maximises our profit. 
With the greedy solution, we achieved a profit of 53.38, whereas with the optimization we could achieve 270 with the same budget.

Optimization helps us to invest our money in the best possible way that yields us higher profit.

### Restricting the number of hotels in each city

Since LQM has decided to buy utmost two hotels in any place, we are adding extra constrints to perform that check while arriving at the optimal solution. This extra checks would yield us lesser profit compared to the optimal solution without these constraints and it is totally sensible.

In [0]:
c2=((city_list[5]=="South Lake Tahoe, California")*x[5] + 
    (city_list[6]=="South Lake Tahoe, California")*x[6] + 
    (city_list[7]=="South Lake Tahoe, California")*x[7] + 
    (city_list[8]=="South Lake Tahoe, California")*x[8] + 
    (city_list[9]=="South Lake Tahoe, California")*x[9] + 
    (city_list[10]=="South Lake Tahoe, California")*x[10] + 
    (city_list[11]=="South Lake Tahoe, California")*x[11] 
   )<=2
c3=((city_list[0]=='Fresno, California')*x[0] + 
    (city_list[3]=='Fresno, California')*x[3] + 
    (city_list[4]=='Fresno, California')*x[4] + 
    (city_list[12]=='Fresno, California')*x[12] 
   )<=2
c4=((city_list[13]=='Los Angeles, California')*x[13] + 
    (city_list[14]=='Los Angeles, California')*x[14] + 
    (city_list[15]=='Los Angeles, California')*x[15] 
   )<=2
con = [c1, c2, c3, c4]

In [0]:
prob = cvx.Problem(obj1, con)
result = prob.solve()

pt = PrettyTable()

print('Hotels that are chosen')

pt.field_names = ["Hotel", "City"]
for i in range(16):
  if (x[i].value > 0.99):
    pt.add_row([hotel_list[i], city_list[i]])

print(pt)

print('Maximum profitability')
print(round(prob.value))

Hotels that are chosen
+-------+------------------------------+
| Hotel |             City             |
+-------+------------------------------+
|   1   |      Eureka, California      |
|   10  | South Lake Tahoe, California |
|   11  | South Lake Tahoe, California |
|   5   |      Fresno, California      |
|   7   |   Los Angeles, California    |
|   8   |   Los Angeles, California    |
+-------+------------------------------+
Maximum profitability
206.0


As expected, the profit from optimal solution with constraints is lesser than the one from optimal solution without constraints. Since it is a business decision that LQM has decided not to buy more that two hotels in a place, we suggest what those hotels should be.

To La Quinta Motors,

Greetings! Congratulations on your recent budget allocation to purchase new hotels.

Please find the data computed from the excel sheet in the above section 'Data computed from input file' which is sorted based on the profitability of the hotels.

As you can see, Hotel #2 is the most profitable one, but also the most expensive one. Since LQM can end up only with $53.38 profit by investing all $10 million, we have built our optimization model catering to your various constraints.

First, this model returned us the following hotels: 5, 10, 11, 12, 13, 15, 16. This combination give the maximum expected total profit within budget. Even though this is a good combination, we found that majority of the hotels are being purchased in South Lake Tahoe, California.

After considering your suggestion to restrict the number of hotels in any place to two, we have arrived at the following hotels: 1, 5, 7, 8, 10, 11. These hotels are present totally in four areas and we strongly believe that this will help LQM expand business in different places.

In addition to our above recommendation, we have a suggestion on your regression model. When investing in hotels, we need more information about the tourists or business meeting happening in that area. This will tell us whether investing in that place would be profitable. 

Additionally, we recommend LQM to first consider buying in a greater number of places to expand the business. If any budget is left over, we can see which place would be better to invest that budget for a second hotel.

This is the best possible suggestion we have arrived using the constraints you have given and your budget. However, if your constraints or budget change, please keep us informed. We can help you anytime!
